In [1]:
import pyspark 
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder \
    .master("local[3]") \
    .appName("Airbnb Listings") \
    .getOrCreate()       

In [ ]:

listings_df = spark.read \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .parquet("listings.parquet")
    


In [6]:
listings_df.select("number_of_reviews", "review_scores_rating","reviews_per_month").show(5) 


+-----------------+--------------------+-----------------+
|number_of_reviews|review_scores_rating|reviews_per_month|
+-----------------+--------------------+-----------------+
|              207|                95.0|             4.07|
|               43|                96.0|             1.48|
|               20|                97.0|             1.15|
|                0|                NULL|             NULL|
|               38|                92.0|             0.89|
+-----------------+--------------------+-----------------+
only showing top 5 rows



In [4]:
#drop unwanted columns 
listings_df = listings_df.select('id','name','summary','longitude','latitude','space','description','instant_bookable','neighborhood_overview','neighbourhood_cleansed','host_id','host_name','host_since',
                 'host_response_time','street', 'zipcode','review_scores_rating','property_type','room_type','accommodates','bathrooms','bedrooms','beds','reviews_per_month','amenities','cancellation_policy','number_of_reviews','price')
print(listings_df.count() , len(listings_df.columns))


3818 28


In [5]:
listings_df = listings_df.withColumn('host_since', listings_df['host_since'].cast('date'))

listings_df = listings_df.withColumn('zipcode', listings_df['zipcode'].cast('int'))

In [6]:
listings_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- space: string (nullable = true)
 |-- description: string (nullable = true)
 |-- instant_bookable: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- neighbourhood_cleansed: string (nullable = true)
 |-- host_id: long (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- review_scores_rating: double (nullable = true)
 |-- property_type: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: long (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: double (nullable = true)
 |-- beds: double (nullable = true)
 |-- reviews_per_month: doubl

In [7]:
# Replace any NAN and null and empty cell in the df to 0
listings_df = listings_df.fillna(0)
# Fill NULL cells with N/A 
listings_df = listings_df.fillna('N/A')


In [8]:
listings_df.show(n=5)

+-------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+----------------+---------------------+----------------------+--------+---------+----------+------------------+--------------------+-------+--------------------+-------------+---------------+------------+---------+--------+----+-----------------+--------------------+-------------------+-----------------+-------+
|     id|                name|             summary|          longitude|          latitude|               space|         description|instant_bookable|neighborhood_overview|neighbourhood_cleansed| host_id|host_name|host_since|host_response_time|              street|zipcode|review_scores_rating|property_type|      room_type|accommodates|bathrooms|bedrooms|beds|reviews_per_month|           amenities|cancellation_policy|number_of_reviews|  price|
+-------+--------------------+--------------------+-------------------+------------------+------------------

In [9]:
# listings_df.select('price').show(5)
# remove $ sign from price column
listings_df= listings_df.withColumn('price', listings_df['price'].substr(2,100).cast('float'))


In [10]:
from pyspark.sql.functions import col
# remove  out listings  with no bedrooms / bathrooms .. price etc

filtered_listings_df = listings_df.filter((col("bedrooms") > 0) 
                                          & (col("bathrooms") > 0) 
                                          & (col("price") > 0) 
                                          &(col('number_of_reviews') > 0) 
                                          & (col('reviews_per_month') > 0) 
                                          & (col('accommodates') > 0) )






In [11]:
# check for nan values
print(filtered_listings_df.filter(filtered_listings_df.bathrooms == 0).count())

0


In [12]:

print(filtered_listings_df.show(n= 5))
print(listings_df.count() , len(listings_df.columns))
print(filtered_listings_df.count() , len(filtered_listings_df.columns))


+-------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+----------------+---------------------+----------------------+--------+---------+----------+------------------+--------------------+-------+--------------------+-------------+---------------+------------+---------+--------+----+-----------------+--------------------+-------------------+-----------------+-----+
|     id|                name|             summary|          longitude|          latitude|               space|         description|instant_bookable|neighborhood_overview|neighbourhood_cleansed| host_id|host_name|host_since|host_response_time|              street|zipcode|review_scores_rating|property_type|      room_type|accommodates|bathrooms|bedrooms|beds|reviews_per_month|           amenities|cancellation_policy|number_of_reviews|price|
+-------+--------------------+--------------------+-------------------+------------------+--------------------+-

In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()

# save the cleaned data to parquet file

filtered_listings_df.write.parquet(os.path.join(current_dir, "cleaned_listings.parquet"), mode='overwrite')

In [18]:
df = spark.read.parquet("cleaned_listings.parquet")
df.show(5)
print(df.count() , len(df.columns))

+-------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+----------------+---------------------+----------------------+--------+---------+----------+------------------+--------------------+-------+--------------------+-------------+---------------+------------+---------+--------+----+-----------------+--------------------+-------------------+-----------------+-----+
|     id|                name|             summary|          longitude|          latitude|               space|         description|instant_bookable|neighborhood_overview|neighbourhood_cleansed| host_id|host_name|host_since|host_response_time|              street|zipcode|review_scores_rating|property_type|      room_type|accommodates|bathrooms|bedrooms|beds|reviews_per_month|           amenities|cancellation_policy|number_of_reviews|price|
+-------+--------------------+--------------------+-------------------+------------------+--------------------+-